In [ ]:
import math

import tabula
import pandas as pd
import numpy as np

In [ ]:
tables = tabula.read_pdf('table.pdf', pages=[4,5,6,7], multiple_tables=True)

# Data Cleaning

In [ ]:
# Manually inspect output tables
for table in tables:
    print(table.info())
    
# Index 0-4 are our main tables. Tabel 5 is smaller table on page 7.

In [ ]:
# Work with the first table
first_table_copy = tables[0].copy()

In [ ]:
first_table_copy.head()

In [ ]:
# See that there are some problems with the header

# Construct a header
data_label = tables[0].iloc[0]
data_type = tables[0].iloc[1]

column_names = []
for i in range(len(data_label)):
    try:
        math.isnan(data_label[i])
        column_names.append(F"{data_type[i]}")
    except TypeError:
        column_names.append(F"{data_label[i]} {data_type[i]}")
        

print(column_names)

In [ ]:
# More cleaning of headers

for i in range(len(column_names)):
    column_name = column_names[i]
    found = column_name.find('exptl - calcd')
    if found != -1:
        column_name = column_name[:found]
    
    column_names[i] = column_name

print(column_names)

In [ ]:
# Now we will get rid of those two rows we don't need

# First drop first two rows
first_table_copy = first_table_copy.drop(index=[0,1])

# Then set the headers
first_table_copy.columns = column_names

In [ ]:
first_table_copy

In [ ]:
# Next split and get rid of value we don't need (we can calculate it ourselves)
for i in range(3,6):
    first_table_copy.iloc[:,i] = first_table_copy.iloc[:, i].str.split().str[0]

In [ ]:
first_table_copy

In [ ]:
# Let's do this in a loop then concatenate the tables.

df = pd.DataFrame()

for table in tables[:-1]:
    table_copy = table.copy()
    
    # Construct a header
    data_label = table_copy.iloc[0]
    data_type = table_copy.iloc[1]

    column_names = []
    for i in range(len(data_label)):
        try:
            math.isnan(data_label[i])
            column_names.append(F"{data_type[i]}")
        except TypeError:
            column_names.append(F"{data_label[i]} {data_type[i]}")
    
    for i in range(len(column_names)):
        column_name = column_names[i]
        found = column_name.find('exptl - calcd')
        if found != -1:
            column_name = column_name[:found]

        column_names[i] = column_name
    
    # First drop first two rows
    table_copy = table_copy.drop(index=[0,1])

    # Then set the headers
    table_copy.columns = column_names
    
    
    # Next split and get rid of value we don't need (we can calculate it ourselves)
    for i in range(3,6):
        table_copy.iloc[:,i] = table_copy.iloc[:, i].str.split().str[0]
    
    df = pd.concat([df, table_copy])
    

In [ ]:
# Set index to molecule numberr
df.set_index('no.', inplace=True)

In [ ]:
# Set 'b' values to NaN
df.replace('b', np.nan, inplace=True)

In [ ]:
# Save as csv
df.to_csv('table_from_paper.csv')

In [ ]:
# Leave this table for later.

tables[4]